#  실습 1: 뉴스기사 불러오기

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
news_rss = requests.get('https://www.hankyung.com/feed/entertainment/SectionRssFeed.do?sectionId=07')
news_rss_soup = BeautifulSoup(news_rss.content, 'xml')
link_list = news_rss_soup.select('item > link')
print("기사 개수:", len(link_list))

title_list = news_rss_soup.select('item > title')
title_list = [title.text for title in title_list]


news_data = []

for link in link_list:
    news_response = requests.get(link.text)
    news_content_soup = BeautifulSoup(news_response.content, 'html.parser')
    news_content = news_content_soup.select_one("div[itemprop=articleBody]")
    news_data.append(news_content)

import pandas as pd
news_df = pd.DataFrame(data={'title': title_list, 'content': news_data})
news_df.to_csv("news.csv", encoding="utf-8-sig", index=False)
print("Save complete")

기사 개수: 50
Save complete


In [5]:
import os
print(os.getcwd())

c:\githome\hipython_rep


# 실습 2: 멀티미디어 데이터 수집하기

In [6]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os

movie_ranking = requests.get("https://www.moviechart.co.kr/rank/realtime/index/image")

image_dir = 'images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
    
pattern = r'[\\/:"*?<>|]'

if movie_ranking.status_code == 200:
    print("영화 정보를 출력합니다.")
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')
    title_list = soup.select(".movieBox-list .movie-title a")
    image_list = soup.select(".movieBox-list .movieBox-item img")
    print(f"수집한 영화 수: {len(title_list)}")
    for title, image in zip(title_list, image_list):
        image_src = image.get('src')
        image_response = requests.get("https://www.moviechart.co.kr" + image_src)
        img = Image.open(BytesIO(image_response.content))
        image_filename = re.sub(pattern, '', title.text)
        img.save(os.path.join(image_dir, image_filename + ".png"))
        print(title.text, )
else:
    print("페이지에 연결할 수 없습니다.")

영화 정보를 출력합니다.
수집한 영화 수: 20
좀비딸
F1 더 무비
전지적 독자 시점
킹 오브 킹스
판타스틱 4: 새로운 출발
악마가 이사왔다
명탐정 코난: 척안의 잔상
발레리나
배드 가이즈 2
베베핀 극장판: 사라진 베베핀과 핑크퐁 대모험
쥬라기 월드: 새로운 시작
노이즈
콘스탄틴
극장판 도라에몽: 진구의 그림이야기
꼬마마법사 주니토니
고고다이노 극장판: 게코도마뱀의 꿈
슈퍼맨
미세리코르디아
강령: 귀신놀이
추적
